# Stimulus Response Analyses

This notebook explores stimulus response analysis to stimuli & events of interest.

Investigated events:



In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from matplotlib.lines import Line2D

from convnwb.io import load_nwbfile

from spiketools.plts.trials import plot_rate_by_time
from spiketools.plts.annotate import add_vlines
from spiketools.plts.style import drop_spines
from spiketools.plts.trials import plot_rasters, create_raster_title
from spiketools.plts.utils import make_axes, save_figure, make_grid, get_grid_subplot
from spiketools.utils.epoch import epoch_spikes_by_event
from spiketools.utils.base import select_from_list
from spiketools.measures.trials import compute_trial_frs
from spiketools.stats.trials import compare_pre_post_activity

In [3]:
import seaborn as sns
sns.set_context('talk')

## Settings

In [4]:
# Define the base data folder
data_folder = '/Users/weijiazhang/Data/Train_NWB'
# Define subject information
session = {
    'experiment' : 'TRAIN',
    'subject' : 'R1219C',
    'session' : 0    
}

In [5]:
# Set a unit of interest to select
uind = 2

In [6]:
# Set the time range to analyze, and pre & post trial windows (in seconds)
trial_range = [-2, 2]
pre_window = [-2, 0]
post_window = [0, 2]

## Load Data

In [7]:
# Load NWB file
nwbfile = load_nwbfile(session, data_folder)

In [13]:
nwbfile.trials

trials pynwb.epoch.TimeIntervals at 0x6373980816
Fields:
  colnames: ['start_time' 'stop_time' 'drive_type' 'feedback_start_time'
 'feedback_stop_time' 'fixation_start_time' 'fixation_stop_time'
 'hold_start_time' 'hold_stop_time' 'movement_start_time'
 'movement_stop_time' 'object' 'object_position' 'response_error'
 'response_position' 'response_time' 'trial_type']
  columns: (
    start_time <class 'hdmf.common.table.VectorData'>,
    stop_time <class 'hdmf.common.table.VectorData'>,
    drive_type <class 'hdmf.common.table.VectorData'>,
    feedback_start_time <class 'hdmf.common.table.VectorData'>,
    feedback_stop_time <class 'hdmf.common.table.VectorData'>,
    fixation_start_time <class 'hdmf.common.table.VectorData'>,
    fixation_stop_time <class 'hdmf.common.table.VectorData'>,
    hold_start_time <class 'hdmf.common.table.VectorData'>,
    hold_stop_time <class 'hdmf.common.table.VectorData'>,
    movement_start_time <class 'hdmf.common.table.VectorData'>,
    movement_sto

In [24]:
# Get object and object position
nwbfile.trials
obj = nwbfile.trials.object[:]
obj_pos = nwbfile.trials.object[:]
# res_pos = nwbfile.trials.response_position[:]
# res_time = nwbfile.trials.response_time[:]



In [25]:
# Check the number of units 
n_units = len(nwbfile.units)

print('Number of units: {}'.format(n_units))
#print('Number to keep:  {}'.format(n_keep))

Number of units: 23


In [26]:
# Get spikes for unit of interest
spikes = nwbfile.units.get_unit_spike_times(uind)

## Subject response time and position

In [27]:
# Get subject response position and time 

res_pos = nwbfile.trials.response_position[:]
res_time = nwbfile.trials.response_time[:]

In [28]:
# Check the number of responses

n_response = len(res_time)
print('Number of responses: {}'.format(n_response ))

Number of responses: 64


In [32]:
# Gather neural data around trials of interest
all_trials_response = epoch_spikes_by_event(spikes, res_time, trial_range)

In [35]:
# Compute averages & statistical comparison of pre / post window firing rates
avg_pre, avg_post, t_val, p_val = compare_pre_post_activity(all_trials_chest, pre_window, post_window)

## Subject response - Split by Stimulus Content

In [36]:
# Create masks to sub-select objects 
barrel_ind = [obj[:]=='b'barrel'' for ind in range(len(movement_end))]


NameError: name 'movement_end' is not defined

In [37]:
obj

array([b'barrel', b'barrel', b'barrel', b'barrel', b'barrel', b'barrel',
       b'box', b'box', b'box', b'box', b'box', b'box', b'box', b'barrel',
       b'box', b'box', b'box', b'barrel', b'barrel', b'barrel', b'box',
       b'barrel', b'box', b'box', b'barrel', b'barrel', b'barrel', b'box',
       b'barrel', b'barrel', b'box', b'box', b'bench', b'bench', b'bench',
       b'bench', b'bench', b'bench', b'desk', b'desk', b'desk', b'desk',
       b'desk', b'desk', b'desk', b'bench', b'bench', b'bench', b'desk',
       b'desk', b'bench', b'desk', b'desk', b'bench', b'bench', b'desk',
       b'desk', b'bench', b'desk', b'desk', b'bench', b'bench', b'bench',
       b'desk'], dtype=object)